In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model

# Load the trained model from HDF5 file
model = load_model('model.hdf5')

def evaluate():
    data = pd.read_csv('sample_input.csv')
    actual_close = np.loadtxt('sample_close.txt')

    pred_close = predict_func(data)

    # Calculation of squared_error
    actual_close = np.array(actual_close)
    pred_close = np.array(pred_close)
    mean_square_error = np.mean(np.square(actual_close - pred_close))

    pred_prev = [data['Close'].iloc[-1]]
    pred_prev.append(pred_close[0])
    pred_curr = pred_close

    actual_prev = [data['Close'].iloc[-1]]
    actual_prev.append(actual_close[0])
    actual_curr = actual_close

    # Calculation of directional_accuracy
    pred_dir = np.array(pred_curr) - np.array(pred_prev)
    actual_dir = np.array(actual_curr) - np.array(actual_prev)
    dir_accuracy = np.mean((pred_dir * actual_dir) > 0) * 100

    print(f'Mean Square Error: {mean_square_error:.6f}\nDirectional Accuracy: {dir_accuracy:.1f}')


def predict_func(data):
    # Columns to process
    columns_to_process = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

    # Iterate over each column
    for column in columns_to_process:
        # Fill null values with linearly interpolated values
        data[column].interpolate(method='linear', inplace=True)

    # Select the input features and target column
    input_features = ['Open', 'Volume', 'High', 'Low', 'Adj Close']
    target_column = 'Close'

    # Extract the input features and target column
    input_data = data[input_features].values
    target_data = data[target_column].values.reshape(-1, 1)

    # Normalize the input features and target column
    scaler = MinMaxScaler(feature_range=(0, 1))
    normalized_input = scaler.fit_transform(input_data)
    normalized_target = scaler.fit_transform(target_data)

    window_size = 25

    # Get the last window of data
    last_window = normalized_input[-window_size:]

    # Reshape the last window to match the expected input shape of the LSTM model
    last_window = np.expand_dims(last_window, axis=0)

    # Make predictions for the last window's values
    predictions = model.predict(last_window)

    # Rescale the predictions to the original scale
    predictions_inverse = scaler.inverse_transform(predictions)

    day = [predictions_inverse[0][0], predictions_inverse[0][1]]

    print(day)
    return day



if __name__ == "__main__":
    evaluate()


1/1 [==============================] - 0s 441ms/step
[7253.8516, 7256.597]
Mean Square Error: 461.294096
Directional Accuracy: 100.0
